In [1]:
import pandas as pd
import numpy as np
import openpyxl
from configparser import ConfigParser, ExtendedInterpolation
import os
import sys
from unidecode import unidecode
import requests
from requests.exceptions import RequestException
from datetime import date
from dateutil import parser

from neo4j import GraphDatabase

from neomodel import config, db, install_labels, install_all_labels, remove_all_labels, drop_constraints
from neomodel import (StructuredNode,  RelationshipTo, RelationshipFrom, StructuredRel)
from neomodel import (StringProperty, IntegerProperty, FloatProperty, ArrayProperty, BooleanProperty, DateProperty, UniqueIdProperty, JSONProperty)
# from neomodel import (AliasProperty, RegexProperty, JSONProperty, DateTimeProperty, DateTimeFormatProperty, PointProperty
from neomodel.exceptions import *

from neomodel.integration.pandas import to_dataframe, to_series
from neomodel.integration.numpy import to_ndarray

from socket import gaierror

from pynsee.utils.init_conn import init_conn
from pynsee.macrodata import  get_dataset_list, get_series_list, get_series, get_series_title, search_macrodata, get_dataset
from pynsee.localdata import get_local_metadata, get_local_data
from pynsee.metadata import get_activity_list
from pynsee.sirene import get_dimension_list, search_sirene, get_sirene_data, get_sirene_relatives
from pynsee.geodata import get_geodata_list, get_geodata
from pynsee import get_file_list, download_file

import eurostat 


In [2]:
from modules.config import session

In [3]:
from modules.neomodel_classes import *
from modules.utils import connect, resumeInstance, get_config, getDS
from modules.constructors import addContract, addEnterprise
from modules.location_management import getDeliveryLocation, getLocationCode

In [6]:
driver.close()

In [4]:
try:
    driver = connect(session, 'self-managed')
except gaierror:
    pass

fichier de configuration C:\Users\Utilisateur\projets\neo4j\config.ini


In [12]:
driver.verify_connectivity()

In [5]:
remove_all_labels()
install_all_labels()



Dropping constraints...
 - Dropping unique constraint and index on label Arrondissement with property arrondissementCode.
 - Dropping unique constraint and index on label Arrondissement with property arrondissementName.
 - Dropping unique constraint and index on label Canton with property cantonCode.
 - Dropping unique constraint and index on label Canton with property cantonName.
 - Dropping unique constraint and index on label Cedex with property codeCedex.
 - Dropping unique constraint and index on label City with property codeCommune.
 - Dropping unique constraint and index on label Contract with property key.
 - Dropping unique constraint and index on label Country with property countryCode.
 - Dropping unique constraint and index on label Departement with property departementCode.
 - Dropping unique constraint and index on label Departement with property departementName.
 - Dropping unique constraint and index on label Enterprise with property titulaireId.
 - Dropping unique cons

In [5]:

decp = pd.read_csv(session['dataset'], sep= ';')

decp.loc[decp.titulaire_denominationSociale_1.isna(), 'titulaire_denominationSociale_1'] = \
decp[decp.titulaire_denominationSociale_1.isna()].titulaire_id_1.apply(lambda x: getDS(str(x)[0:14]))

decp.loc[decp.titulaire_typeIdentifiant_2.notna() & decp.titulaire_denominationSociale_2.isna(), 'titulaire_denominationSociale_2'] = \
decp[decp.titulaire_typeIdentifiant_2.notna() & decp.titulaire_denominationSociale_2.isna()].titulaire_id_2.apply(lambda x: getDS(str(x)[0:14]))

decp.loc[decp.titulaire_typeIdentifiant_3.notna() & decp.titulaire_denominationSociale_3.isna(), 'titulaire_denominationSociale_3'] = \
decp[decp.titulaire_typeIdentifiant_3.notna() & decp.titulaire_denominationSociale_3.isna()].titulaire_id_3.apply(lambda x: getDS(str(x)[0:14]))


This function may return personal data, please check and comply with the legal framework relating to personal data protection !
API query number limit reached - function might be slowed down


In [56]:
decp[decp.duplicated(subset = ['titulaire_id_1', 'id', 'objet', 'dateNotification', 'booleanModification'], keep= False)][['titulaire_id_1', 'dureeMois', 'id', 'dateNotification', 'objet', 'montant', 'acheteur.id', 'lieuExecution.code', 'booleanModification']].sort_values(by= 'id')

,titulaire_id_1,dureeMois,id,dateNotification,objet,montant,acheteur.id,lieuExecution.code,booleanModification
71090,8.837142e+13,3,202000604800,2023-04-24,"LA BRIDOIRE Route de Vergenucle, Travaux, MS",247962.77,25730223200034,73290,0
32822,8.837142e+13,3,202000604800,2023-04-24,"LA BRIDOIRE Route de Vergenucle, Travaux, MS",285839.10,25730223200034,73290,0
41031,3.928076e+13,11,2021-03-04-0100,2023-03-21,Lot 4 - Charpente,296081.31,21850071800018,85220,0
57408,3.928076e+13,11,2021-03-04-0100,2023-03-21,Lot 4 - Charpente,296081.35,21850071800018,85220,0
55948,4.237536e+13,3,202115-16-19L300,2023-01-25,TRANCHES 15-16-19 LOT 3 MONESTIER,220355.56,20009657600026,24100,0
...,...,...,...,...,...,...,...,...,...
48625,4.803660e+13,12,2023TONELGEOM00,2023-05-10,"Aménagement Lotissement les Tonnelles ""Mission...",65340.00,20005770100019,85019,0
65365,9.173205e+13,2,2023TR02-100,2023-01-18,PROJET D'AMENAGEMENT DE LA RUE JEAN MOULIN ET ...,129988.56,21340146600015,34400,0
41630,9.173205e+13,2,2023TR02-100,2023-01-18,PROJET D'AMENAGEMENT DE LA RUE JEAN MOULIN ET ...,108232.80,21340146600015,34400,0
18798,4.297233e+13,5,2023TVX LA POS00,2023-02-22,Travaux de réseaux la Poste,452270.41,21730067200011,73130,0


In [57]:
decp[decp.duplicated(subset = ['titulaire_id_1', 'id', 'montant', 'objet', 'acheteur.id', 'booleanModification'], keep= False)][['titulaire_id_1', 'dureeMois', 'id', 'dateNotification', 'objet', 'montant', 'acheteur.id', 'lieuExecution.code', 'booleanModification']].sort_values(by= 'id')

,titulaire_id_1,dureeMois,id,dateNotification,objet,montant,acheteur.id,lieuExecution.code,booleanModification
77030,3.240959e+13,36,202201301FS09000,2023-03-02,LOT 1 NETTOYAGE DES LOCAUX - BATIMENTS ADMINIS...,248296.53,21910340500128,91090,0
55305,3.240959e+13,36,202201301FS09000,2023-02-17,LOT 1 NETTOYAGE DES LOCAUX - BATIMENTS ADMINIS...,248296.53,21910340500128,91090,0
84593,7.512698e+13,8,202213DSTUP00,2023-04-04,DSTUP - MOE PLAN D'EAU,65000.00,21140258100014,14700,0
9321,7.512698e+13,8,202213DSTUP00,2023-03-02,DSTUP - MOE PLAN D'EAU,65000.00,21140258100014,14700,0
37158,8.389269e+13,5,2023-0200,2023-07-11,EXTENSION CABANE PASTORALE ALPAGE DE PEYNIN LO...,58322.30,21050003900016,05470,0
59750,8.389269e+13,5,2023-0200,2023-05-11,EXTENSION CABANE PASTORALE ALPAGE DE PEYNIN LO...,58322.30,21050003900016,05470,0
44745,3.232226e+13,5,202301-0200,2023-06-05,LOT No2 PRESTATION DE CONCEPTION REALISATION E...,97799.00,21650395300012,65270,0
52058,3.232226e+13,5,202301-0200,2023-06-16,LOT No2 PRESTATION DE CONCEPTION REALISATION E...,97799.00,21650395300012,65270,0
24011,3.099308e+13,4,202312700,2023-07-03,Aménagement de dispositifs de franchissement p...,55863.49,22220001600327,22,0
29986,3.099308e+13,4,202312700,2023-07-11,Aménagement de dispositifs de franchissement p...,55863.49,22220001600327,22,0


In [7]:
decp[1:1000].apply(addContract, axis= 1)

1      None
2      None
3      None
4      None
5      None
       ... 
995    None
996    None
997    None
998    None
999    None
Length: 999, dtype: object

In [ ]:
decp[7700:].apply(addContract, axis= 1)

XXXXXXXXXX type non trouvé XXXXXXXX collectivite doutre-mer a statut particulier FR-NC
XXXXXXXXXX type non trouvé XXXXXXXX collectivite doutre-mer a statut particulier FR-NC
XXXXXXXXXX type non trouvé XXXXXXXX collectivite doutre-mer a statut particulier FR-NC


In [26]:
decp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95635 entries, 0 to 95634
Data columns (total 53 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   titulaire_denominationSociale_1     95615 non-null  object 
 1   titulaire_id_1                      95213 non-null  float64
 2   titulaire_typeIdentifiant_1         95635 non-null  object 
 3   titulaire_denominationSociale_2     7366 non-null   object 
 4   titulaire_id_2                      7330 non-null   float64
 5   titulaire_typeIdentifiant_2         7364 non-null   object 
 6   titulaire_denominationSociale_3     3392 non-null   object 
 7   titulaire_id_3                      3376 non-null   float64
 8   titulaire_typeIdentifiant_3         3393 non-null   object 
 9   procedure                           91173 non-null  object 
 10  nature                              95507 non-null  object 
 11  codeCPV                             95631

In [65]:
decp[decp.booleanModification == 0][decp.duplicated(subset= ['titulaire_id_1', 'id', 'objet', 'montant', 'acheteur.id'], keep= False)][['id', 'objet', 'montant']]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_27716\1836229113.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  decp[decp.booleanModification == 0][decp.duplicated(subset= ['titulaire_id_1', 'id', 'objet', 'montant', 'acheteur.id'], keep= False)][['id', 'objet', 'montant']]


,id,objet,montant
6362,20232300100100,PRESTATION CURAGE RES ET OUVRAGES,500000.00
8228,202323MP02400100,AMENAGEMENT DES ABORDS DU PORT,573960.60
9321,202213DSTUP00,DSTUP - MOE PLAN D'EAU,65000.00
21243,202323MP02200100,AMENAGEMENT DES ABORDS DU PORT,2516155.05
24011,202312700,Aménagement de dispositifs de franchissement p...,55863.49
29986,202312700,Aménagement de dispositifs de franchissement p...,55863.49
35728,202323MP02300100,AMENAGEMENT DES ABORDS DU PORT,724106.70
36274,20234200,RESAH 2019-063 - Matériel informatique - Lot 3...,3000000.00
37158,2023-0200,EXTENSION CABANE PASTORALE ALPAGE DE PEYNIN LO...,58322.30
44745,202301-0200,LOT No2 PRESTATION DE CONCEPTION REALISATION E...,97799.00


In [59]:
dup[dup.booleanModification == 0]

,id,objet,montant,booleanModification
31,2023F2301700,SCHEMA DIRECTEUR ASSAINISSEMENT,214950.00,0
582,2023GC0100,Acquisition ou location avec option d'achat d'...,50000.00,0
615,202323VV006B00,TRAVAUX DE REHABILITATION DE L ANCIEN HOTEL DE...,6386583.98,0
649,2023M2305700,M23057 - LOT 5 - TECHNIQUES - REHABILITATION D...,1318522.99,0
747,20221183500,MO RENOV RESIDENCE BECART AU CREPS WATTIGNIES,303600.00,0
...,...,...,...,...
95313,2023MA23120100,AC FOURNITURE - MISE EN OEUVRE DE SPECTACLE PY...,74500.00,0
95373,2022MAT22120100,LOT1 VRD/GROS OEUVRE/MACONNERIE CONSTR.CASIERS...,83864.31,0
95431,2023P000100,LOCATION ET MAINTENANCE DUNE BENNE A ORDURES M...,83760.00,0
95442,2023carbulot100,Marché carburant 2023_2027 lot 1 GAZOLE,832255.09,0


In [46]:
decp[decp.booleanModification == 1].booleanModification.sum()

4414

In [205]:

# codevalues = ['GF', 'FRB06', 'RE', 'FRJ11', 'GP', 'MQ', 'FRB02', 'YT', 'FRE22', 'MF', 'FRK26', 'FRL03', 'FRF22', 'GARD', 'FRG05', 'PM']
codevalues = ['FRB06','FRJ11','FRB02', 'FRE22', 'FRK26', 'FRL03', 'FRF22', 'FRG05']
codevalues = ['FR-COR','FR-GP','FR-GF', 'FR-RE', 'FR-MQ', 'FR-YT', 'FR-MF']
codevalues = ['NOYON', 'COMM', 'DEP']
codevalues = ['99100']

for value in codevalues:
    codes = {}
    codes['code'] = value
    codes['type'] = 'code pays'
    codes['communeBuyer'] = 'BOS MORIN'
    codes['final'] = ''

    print(getLocationCode(codes))
    # print(getLocationCode(codes))
    # print(checkNutsCode(codes))

{'code': '99100', 'type': 'cedex', 'communeBuyer': 'BOS MORIN', 'final': 'cedex', 'subtype': 'cedex', 'country': 'FR', 'region': '', 'cedex': '99100', 'commune': '', 'postal': '', 'region_name': '', 'commune_name': '', 'departement': '', 'departement_name': ''}


In [21]:
codevalues = ['19100', '78297', '5711', 'FR101', '2A', '2B', '30',
              'FR', '28', '142', 'FR-07', 'FR-6AE']
coderesults = ['code postal', 'code commune', 'code canton', 'nuts', 'code region', 'code region', 'code region',
               'code pays', 'code region', 'code arrondissement', 'iso', 'iso']

for value in codevalues:
    codes = {}
    codes['code'] = value
    codes['type'] = 'code postal'
    codes['region'] = ''
    codes['region_name'] = ''
    codes['departement'] = ''
    codes['departement_name'] = ''
    codes['final'] = ''
    codes['nom'] = ''
    codes['commune'] = ''
    codes['commune_name'] = ''
    codes['postal'] = ''
    codes['cedex'] = ''
    codes['country'] = ''
    codes['subtype'] = ''
    codes['communeBuyer'] = ''
    print(getLocationCode(codes))


{'code': '19100', 'type': 'code commune', 'region': '75', 'region_name': 'NOUVELLE AQUITAINE', 'departement': '19', 'departement_name': 'CORREZE', 'final': 'commune', 'nom': '', 'commune': '19031', 'commune_name': 'BRIVE LA GAILLARDE', 'postal': '19100', 'cedex': '', 'country': '', 'subtype': 'commune', 'communeBuyer': ''}
{'code': '78297', 'type': 'cedex', 'region': '11', 'region_name': 'ILE DE FRANCE', 'departement': '78', 'departement_name': 'YVELINES', 'final': 'cedex', 'nom': '', 'commune': '', 'commune_name': '', 'postal': '', 'cedex': '78297', 'country': '', 'subtype': 'cedex', 'communeBuyer': ''}
{'code': '5711', 'type': 'code canton', 'region': '44', 'region_name': 'GRAND EST', 'departement': '57', 'departement_name': 'MOSELLE', 'final': 'canton', 'nom': '', 'commune': '', 'commune_name': '', 'postal': '', 'cedex': '', 'country': 'FR', 'subtype': 'canton', 'communeBuyer': '', 'canton_name': 'METZ 1', 'canton': '5711'}
{'code': 'FR101', 'type': 'code region', 'region': '0', 're

In [157]:
siret = '21971123100072'
dfBuyer = search_sirene(variable = ['siret'], number= 1,
                               pattern = [siret], phonetic_search = False, legal=True, closed=True)
dfBuyer = dfBuyer.iloc[0, :]
dfBuyer = dfBuyer.to_frame().transpose()
buyerName = str(*dfBuyer.denominationUniteLegale)

df.info()

<class 'pynsee.sirene.SireneDataFrame.SireneDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 92 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   siren                                           1 non-null      object 
 1   nic                                             1 non-null      object 
 2   siret                                           1 non-null      object 
 3   dateDebut                                       1 non-null      object 
 4   dateCreationEtablissement                       1 non-null      object 
 5   dateCreationUniteLegale                         1 non-null      object 
 6   dateFin                                         0 non-null      object 
 7   denominationUniteLegale                         1 non-null      object 
 8   nomUniteLegale                                  0 non-null      object 
 9   prenomUsuelUniteLegale       

In [158]:
communeBuyer = df.codeCommuneEtablissement.values.all()
communeBuyer

'97701'

In [159]:
buyerName

'COLLECTIVITE DE SAINT BARTHELEMY'

In [150]:
df.code

,siren,nic,siret,dateDebut,dateCreationEtablissement,dateCreationUniteLegale,dateFin,denominationUniteLegale,nomUniteLegale,prenomUsuelUniteLegale,...,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement
0,219711231,00072,21971123100072,2007-02-21,2007-02-21,1981-06-23,None,COLLECTIVITE DE SAINT BARTHELEMY,None,None,...,None,None,None,None,None,None,None,None,None,None


In [164]:
driver.close()